In [1]:
# --- 导入与设置 ---
import numpy as np
import pandas as pd
import gc
import os

# PyTorch and Dataloader
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
# Scikit-learn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

# AnnData for loading
import anndata as ad
from scipy.sparse import issparse

# --- 全局配置 ---
name = 'multi' # 本次处理的数据集
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

def neg_correlation_score(y_pred: torch.Tensor, y_true: torch.Tensor, eps: float = 1e-8) -> torch.Tensor:
    """
    Row-wise Pearson correlation (y_true vs y_pred), then return negative mean.
    Shapes: [batch, dim] (e.g., dim = #targets per sample)
    """
    # center per row
    x = y_true - y_true.mean(dim=1, keepdim=True)
    y = y_pred - y_pred.mean(dim=1, keepdim=True)

    # numerator & denominator
    num = (x * y).sum(dim=1)                               # [batch]
    den = torch.sqrt((x.square().sum(dim=1) + eps) *
                     (y.square().sum(dim=1) + eps))        # [batch]

    r = num / den                                          # row-wise corr
    return -r.mean()                                       # negative average correlation

def zscore_col(x):
    """Applies z-score normalization to each COLUMN of a dense numpy array."""
    mean = np.mean(x, axis=0, keepdims=True)
    std = np.std(x, axis=0, keepdims=True)
    std[std == 0] = 1 # Avoid division by zero
    return (x - mean) / std

Using device: cuda


In [2]:
# --- Cell2: Data Reduction (Memory-Efficient) ---
import anndata as ad
import pandas as pd
import numpy as np
from scipy import sparse # Import sparse explicitly
from scipy.sparse import issparse
import gc
from sklearn.preprocessing import normalize

# --- a) Load sparse data from .h5ad files using backed mode ---
print("Loading sparse data from .h5ad files in backed mode...")
name = 'cite' # Assuming 'name' is defined
PATH_TRAIN_INP = f"train_{name}_inputs.h5ad"
PATH_TRAIN_TGT = f"train_{name}_targets.h5ad"
PATH_TEST_INP  = f"test_{name}_inputs.h5ad"

# MODIFICATION: Use backed='r' to avoid loading the whole file into memory
# We now load the AnnData objects in read-only backed mode.
adata_train_inp = ad.read_h5ad(PATH_TRAIN_INP, backed='r')
adata_train_tgt = ad.read_h5ad(PATH_TRAIN_TGT, backed='r')
adata_test_inp = ad.read_h5ad(PATH_TEST_INP, backed='r')

# Get pointers to the sparse matrices on disk
# These are not in memory yet
train_X_sparse = adata_train_inp.X
test_X_sparse = adata_test_inp.X
train_y_sparse = adata_train_tgt.X

print(f"Initial sparse train_X shape: {train_X_sparse.shape}")
print(f"Initial sparse train_y shape: {train_y_sparse.shape}")


# --- Helper function for chunk-wise processing ---
def scale_by_row_median_chunked(X: sparse.spmatrix, chunk_size: int = 1000):
    """
    Scales a sparse matrix by the median of non-zero values in each row,
    processing the matrix in chunks to save memory.
    X can be a view from a backed AnnData object.
    """
    # Create an empty sparse matrix with the same format to store results
    # Using lil_matrix is efficient for row-by-row construction
    X_scaled = sparse.lil_matrix(X.shape, dtype=np.float32)
    
    for i in range(0, X.shape[0], chunk_size):
        # Load one chunk into memory
        chunk = X[i:i+chunk_size].tocsr() # .tocsr() is a good format for row operations
        
        # Process this smaller chunk
        for j in range(chunk.shape[0]):
            row_data = chunk[j].data
            if len(row_data) > 0:
                median = np.median(row_data)
                if median > 0:
                    chunk[j].data /= median
        
        # Assign the processed chunk to the result matrix
        X_scaled[i:i+chunk_size] = chunk
        
        # Optional: print progress
        if (i // chunk_size) % 10 == 0:
            print(f"  Processed {i + chunk_size} / {X.shape[0]} rows...")
            
    # Convert from lil_matrix to csr_matrix for efficient arithmetic and storage
    return X_scaled.tocsr()


# --- Input Matrix Preprocessing (train_X, test_X) ---
print("\n--- Preprocessing Input Matrices (X) in chunks... ---")
# 1. Divide by row-wise non-zero median
train_X = scale_by_row_median_chunked(train_X_sparse)
test_X = scale_by_row_median_chunked(test_X_sparse)
print("Finished preprocessing X.")


# --- Target Matrix Preprocessing (train_y) ---
print("\n--- Preprocessing Target Matrix (y) ... ---")
# 1. Scale target by row-wise non-zero median (also chunked)
train_y = scale_by_row_median_chunked(train_y_sparse)

# 2. Apply log1p on sparse data's non-zero elements
print("Step 2: Applying log1p on sparse data...")
train_y.data = np.log1p(train_y.data)

# 3. Row-wise L2 normalization (sklearn's normalize is sparse-friendly)
print("Step 3: Row-wise L2 normalization...")
if issparse(train_y):
    # work on CSR for axis=1 ops
    train_y = train_y.tocsr(copy=True)
    # replace NaN/Inf in sparse data buffer
    np.nan_to_num(train_y.data, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    train_y.eliminate_zeros()  # purge zeros we just created
else:
    # in-place sanitize for dense
    np.nan_to_num(train_y, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
train_y = normalize(train_y, norm='l2', axis=1, copy=False)
train_y = train_y.astype(np.float32)
print("Finished preprocessing y.")


# --- Clean up and final check ---
print("\nCleaning up memory...")
# The backed anndata objects don't hold much memory, but we can delete them.
del adata_train_inp, adata_train_tgt, adata_test_inp
del train_X_sparse, train_y_sparse, test_X_sparse
gc.collect()

print("\nPreprocessing complete. Final shapes:")
print(f"train_X: {train_X.shape} (sparse: {issparse(train_X)})")
print(f"train_y: {train_y.shape} (sparse: {issparse(train_y)})")
print(f"test_X: {test_X.shape} (sparse: {issparse(test_X)})")

Loading sparse data from .h5ad files in backed mode...
Initial sparse train_X shape: (64074, 15435)
Initial sparse train_y shape: (64074, 140)

--- Preprocessing Input Matrices (X) in chunks... ---
  Processed 1000 / 64074 rows...
  Processed 11000 / 64074 rows...
  Processed 21000 / 64074 rows...
  Processed 31000 / 64074 rows...
  Processed 41000 / 64074 rows...
  Processed 51000 / 64074 rows...
  Processed 61000 / 64074 rows...
  Processed 1000 / 48663 rows...
  Processed 11000 / 48663 rows...
  Processed 21000 / 48663 rows...
  Processed 31000 / 48663 rows...
  Processed 41000 / 48663 rows...
Finished preprocessing X.

--- Preprocessing Target Matrix (y) ... ---
  Processed 1000 / 64074 rows...
  Processed 11000 / 64074 rows...
  Processed 21000 / 64074 rows...
  Processed 31000 / 64074 rows...
  Processed 41000 / 64074 rows...
  Processed 51000 / 64074 rows...
  Processed 61000 / 64074 rows...
Step 2: Applying log1p on sparse data...
Step 3: Row-wise L2 normalization...


d:\temp\ipykernel_14236\187267399.py:82: RuntimeWarning: invalid value encountered in log1p
  train_y.data = np.log1p(train_y.data)


Finished preprocessing y.

Cleaning up memory...

Preprocessing complete. Final shapes:
train_X: (64074, 15435) (sparse: True)
train_y: (64074, 140) (sparse: True)
test_X: (48663, 15435) (sparse: True)


In [3]:
# --- Cell 3: Dataset Class (Corrected) ---
from torch.utils.data import Dataset
from scipy.sparse import issparse

class Dataset(Dataset):
    def __init__(self, features, targets=None):
        self.features = features
        self.targets = targets # Changed name to self.targets for clarity
        self.is_train = targets is not None

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        # Process feature
        feature = self.features[idx]
        if issparse(feature):
            feature = feature.toarray().squeeze()
        
        # Using torch.from_numpy is generally preferred over torch.tensor for numpy arrays
        # to avoid data copy where possible.
        feature_tensor = torch.from_numpy(feature).float()
        
        if self.is_train:
            # Process target
            target = self.targets[idx]
            if issparse(target):
                # The crucial fix: convert the sparse target row to a dense array first
                # 关键修正：首先将稀疏的 target 行转换为密集数组
                target = target.toarray().squeeze()
            
            target_tensor = torch.from_numpy(target).float()
            
            return feature_tensor, target_tensor
        else:
            return feature_tensor

In [4]:
# --- Cell 4 : Model Architecture ---

import torch
import torch.nn as nn
import torch.nn.functional as F

class VAETransformer(nn.Module):
    """
    一个集成了 VAE 和 Transformer 的模型。
    工作流程:
    1. VAE Encoder: (batch, num_features) -> mu, log_var (batch, latent_dim)
    2. Reparameterization Trick: 从 mu, log_var 采样 -> z (batch, latent_dim)
    3. Transformer: 将 z 视为长度为1的序列进行处理 -> (batch, latent_dim)
    4. VAE Decoder: (batch, latent_dim) -> (batch, target_dim)
    """
    def __init__(self, num_features, target_dim, latent_dim=256, hidden_dim=1024,
                 nhead=8, num_encoder_layers=3, dim_feedforward=1024, dropout=0.1):
        """
        初始化 VAETransformer 模型。

        参数:
            num_features (int): 输入特征维度。
            target_dim (int): 输出目标维度
            latent_dim (int): VAE 的隐空间维度，同时也是 Transformer 的 d_model。
            hidden_dim (int): Encoder 和 Decoder 中间隐藏层的维度。
            nhead (int): Transformer 的头数。
            num_encoder_layers (int): Transformer Encoder 的层数。
            dim_feedforward (int): Transformer 前馈网络的维度。
            dropout (float): Dropout 比例。
        """
        super().__init__()
        self.latent_dim = latent_dim

        # --- 1. VAE Encoder ---
        self.encoder = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout)
        )
        self.fc_mu = nn.Linear(hidden_dim // 2, latent_dim)
        self.fc_log_var = nn.Linear(hidden_dim // 2, latent_dim)

        # --- 2. Transformer Encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=latent_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            num_layers=num_encoder_layers
        )

        # --- 3. VAE Decoder ---
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, target_dim)
        )

    def reparameterize(self, mu, log_var):
        """
        重参数化技巧，使得采样过程可导。
        在评估模式下，直接返回 mu 以获得确定性输出。
        """
        if not self.training:
            return mu
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        """
        前向传播。
        x 的形状: (batch_size, num_features)
        """
        # 编码
        h = self.encoder(x)
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)

        # 采样隐向量 z
        z = self.reparameterize(mu, log_var)

        # Transformer 处理
        # 增加序列维度: (batch, latent_dim) -> (batch, 1, latent_dim)
        z_seq = z.unsqueeze(1)
        transformer_out = self.transformer_encoder(z_seq)
        # 去除序列维度: (batch, 1, latent_dim) -> (batch, latent_dim)
        processed_z = transformer_out.squeeze(1)

        # 解码
        reconstruction = self.decoder(processed_z)

        # 返回重构结果和分布参数（用于计算损失）
        return reconstruction, mu, log_var

In [5]:
# --- Cell 5: Loss function ---

class CombinedLoss(nn.Module):
    """
    组合损失函数
    """
    def __init__(self):
        super().__init__()
        self.mse_loss = nn.MSELoss()
        self.corr_loss = neg_correlation_score

    def forward(self, pred, true, mu, log_var):

        loss1 = self.mse_loss(pred, true)
        loss2 = self.corr_loss(pred, true)
        loss3 = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        loss = loss1 + loss2 + loss3
        return loss

In [6]:
#--- Cell 6: Training function ---
def train_one_fold(
    train_loader,
    valid_loader,
    model,
    optimizer,
    loss_fn,
    train_params,
    n_fold,
    name
):
    """
    Trains the dual-model setup for a single cross-validation fold.
    """
    # Initialize variables to track the best model
    best_val_loss = float('inf')
    model_path = f"model_{name}_fold_{n_fold+1}.pth"

    # Main training loop for the specified number of epochs
    for epoch in range(train_params['epochs']):
        # --- Training Phase ---
        model.train()
        for features, targets in train_loader:
            # Move data to the GPU
            features, targets = features.to(DEVICE), targets.to(DEVICE)
            
            # Reset gradients
            optimizer.zero_grad()
            
            # Forward pass through the first model
            preds, mu, log_var = model(features)
            
            # Calculate the combined loss
            loss = loss_fn(preds, targets, mu, log_var)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # --- Validation Phase ---
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for features, targets in valid_loader:
                # Move data to the GPU
                features, targets = features.to(DEVICE), targets.to(DEVICE)
                
                # Forward pass
                preds, mu, log_var = model(features)

                # Calculate loss for the validation batch
                loss = loss_fn(preds, targets, mu, log_var)
                val_loss += loss.item() * len(features)
        
        val_loss /= len(valid_loader.dataset)
        
        # Check if the model has improved and save the best version
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # Save the state dictionaries of both models
            torch.save(model.state_dict(), model_path)
            
        # Print progress periodically
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{train_params['epochs']}, Val Loss: {val_loss:.4f}")
    
    print(f"Best validation loss for fold {n_fold+1}: {best_val_loss:.4f}")
    # Return the path to the best saved model for this fold
    return model_path

In [ ]:
#--- Cell 7: Train and Validation
MODEL_PARAMS = { # Parameters for the GRUBackbone
    'num_features': train_X.shape[1], 
    'target_dim': train_y.shape[1],
    'latent_dim': 1024,
    'hidden_dim': 1024,
    'nhead': 16,
    'num_encoder_layers': 5,
    'dim_feedforward': 1024,
    'dropout': 0.1,
}
TRAIN_PARAMS = {'batch_size': 512, 'epochs': 50, 'lr': 5e-3, 'loss_alpha': 0.5}
folds = KFold(n_splits=5, shuffle=True, random_state=1337)

# --- Initialize Prediction Arrays ---
oof_preds = np.zeros(train_y.shape, dtype=np.float32)
sub_preds = np.zeros((test_X.shape[0], train_y.shape[1]), dtype=np.float32)
test_loader = DataLoader(Dataset(test_X, np.zeros((test_X.shape[0], 128))), 
                         batch_size=TRAIN_PARAMS['batch_size'] * 2)


# --- Main CV Loop ---
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_X)):
    print(f"\n===== Processing Fold {n_fold+1}/{folds.get_n_splits()} =====")
    
    # 1. Slice data for the current fold
    X_train_fold, X_valid_fold = train_X[train_idx], train_X[valid_idx]
    y_full_train, y_full_valid = train_y[train_idx], train_y[valid_idx]
    
    # 2. Prepare DataLoaders, Models, Optimizer, and Loss for the fold
    train_dataset = Dataset(X_train_fold, y_full_train)
    valid_dataset = Dataset(X_valid_fold, y_full_valid)
    train_loader = DataLoader(train_dataset, batch_size=TRAIN_PARAMS['batch_size'], shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=TRAIN_PARAMS['batch_size'] * 2)
    
    model = VAETransformer(**MODEL_PARAMS).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=TRAIN_PARAMS['lr'])
    loss_fn = CombinedLoss().to(DEVICE)

    # 3. Call the training function for this fold
    best_model_path = train_one_fold(
        train_loader, valid_loader, model, optimizer, loss_fn,
        TRAIN_PARAMS, n_fold, f"{name}_multitask"
    )
    
    # 4. Load the best model and perform predictions
    best_model = torch.load(best_model_path)
    model.load_state_dict(best_model)
    model.eval()
    
    # 4a. OOF predictions
    val_preds_list = []
    with torch.no_grad():
        for features, _ in valid_loader:
            preds, _, _ = model(features.to(DEVICE))
            val_preds_list.append(preds.cpu().numpy())
    oof_preds[valid_idx] = np.concatenate(val_preds_list)

        # 4b. Test set predictions (accumulated)
    start_idx = 0
    with torch.no_grad():
        for features, _ in test_loader:
            preds, _, _ = model(features.to(DEVICE))
            preds = preds.detach().cpu().numpy()
            end_idx = start_idx + preds.shape[0]
            sub_preds[start_idx:end_idx, :] += preds / folds.get_n_splits()
            start_idx = end_idx
            
    # 5. Clean up memory for the next fold
    del model, X_train_fold, y_full_train, X_valid_fold, y_full_valid
    gc.collect()
    torch.cuda.empty_cache()


# --- save results ---
np.save(f'sub_preds_{name}.npy', sub_preds)
print('Prediction on test dataset had been saved, shape:', sub_preds.shape)
    


SyntaxError: invalid syntax (2451408440.py, line 17)